### Import modules

In [1]:
from itertools import count

from scrapper.print_better_dic import print_dict
from scrapper.extract_artcls_data import current_artcls, artcl_data, artcl_content
from scrapper.location_detector import detect_location


### Save url containing articles

In [2]:
# Base url
news_url= "https://www.elnuevodia.com/noticias/seguridad"

### Construct data_handler dictionary

In [3]:
data_handler= {}

articles= current_artcls(news_url)

for _ in zip(count(start=1, step=1), articles):
    data_handler[str(_[0])]= artcl_data(article= _[1])
    data_handler[str(_[0])].update(artcl_content(article_data= data_handler[str(_[0])]))
    data_handler[str(_[0])].update(detect_location(article_data= data_handler[str(_[0])]))

### Print data_handler

In [4]:
print(print_dict(data_handler))

1: 
    headline: La Policía arresta a sospechoso de asaltar a octogenaria en Bayamón
    url: https://www.elnuevodia.com/noticias/seguridad/notas/la-policia-arresta-a-sospechoso-de-asaltar-a-una-octogenaria-en-bayamon/
    author: Por Andrea Guemárez Soto
    headline_sub: El incidente fue captado en una cámara de seguridad ubicada en el estacionamiento de un supermercado
    date_time: 2024-07-17 18:57:00
    content: Tras entregarse a las autoridades, agentes del Negociado de la Policía de Aguas Buenas arrestaron en la tarde del miércoles a un individuo sospechoso de, presuntamente, asaltar a una adulta mayor de 81 años en un incidente registrado ayer, martes, en el estacionamiento de un supermercado en Bayamón, confirmó la Uniformada. Según supo El Nuevo Día, el sujeto, cuya identidad, de momento, no fue revelada, se entregó en un cuartel de Aguas Buenas luego de que la Policía “ejerciera presión” mediante una búsqueda encabezada por diversas divisiones de la Uniformada, entre esta

In [ ]:
# Import module 
import mysql.connector

# Make the connection to databese
connection = mysql.connector.connect(user="root",
                                     password="root",
                                     host="mysql",
                                     port="3306",
                                     database="db")
print("DB Connected!")

# Create cursor obj
cursor= connection.cursor(buffered= True)

In [ ]:
# Create variable with MySQL query to insert data into articles_data table
data_query = """
INSERT INTO articles_data (headline,
                           url,
                           author,
                           date_time,
                           municipality)
VALUES (%(headline)s, %(url)s, %(author)s, %(date_time)s, %(municipality)s) 
"""

import json
# Commented below for testing
for _, __ in data_handler.items():
    data = {
        "headline": __["headline"],
        "url": __["url"],
        "author": __["author"],
        "date_time": __["date_time"],
        # municipality list is first transformed into json string,
        # then its encoded to bytes, and finally the bytes are encoded to base64
        # to decode use base64.b64decode(<encoded base64 list>)).decode('utf-8')
        # base64.b64encode(json.dumps(__["municipality"]).encode('utf-8'))
        "municipality": json.dumps(__["municipality"])
        }
    # Execute the query with data provided
    cursor.execute(data_query, data)

    # Make sure data is committed to the database
    # Commented for testing but must be done when not testing to make sure data is saved in the database table
    # connection.commit()

### Store data_handler data into database table

In [ ]:
# query= """SELECT municipality FROM articles_data"""
# cursor.execute(query)
# for _ in cursor:
#     try:
#         print(json.loads(_[0])[0])
#     except:
#         print("Municipality not detected!")

In [ ]:
# # Import module 
# import mysql.connector

# # Make the connection to databese
# connection= mysql.connector.connect(user= "root", password= "root", host= "mysql", port= "3306", database= "db")
# print("DB Connected!")

# # Create cursor obj
# cursor= connection.cursor(buffered= True)

In [ ]:
# query= """SHOW TABLES"""
# cursor.execute(query)
# for i in cursor:
#     print(i)

In [ ]:
# # Import module 
# import mysql.connector

# # Make the connection to databese
# connection= mysql.connector.connect(user= "root", password= "root", host= "mysql", port= "3306", database= "db")
# print("DB Connected!")

# # Create cursor obj
# cursor= connection.cursor(buffered= True)

In [ ]:
# # Create variable with MySQL query to insert data into articles_data table
# data_query= """
# INSERT INTO articles_data (headline,
#                            url,
#                            author,
#                            date,
#                            time,
#                            location)
# VALUES (%(headline)s, %(url)s, %(author)s, %(date)s, %(time)s, %(pueblos)s) 
# """

# # Get the data from data_handler and format into dict so MySQL can understand it
# data= {"headline": data_handler["2"]["headline"],
#        "url": data_handler["2"]["url"],
#        "author": data_handler["2"]["author"],
#        "date": data_handler["2"]["date"],
#        "time": data_handler["2"]["time"],
#        "pueblos": data_handler["2"]["pueblos"][0]} # This must be rewritten in a way that MySQL can store lists in the field,
#                                                    # right now I just picked the first item from the returned list

# # Execute the query with data provided
# cursor.execute(data_query, data)

# # Make sure data is committed to the database
# # Commented for testing but must be done when not testing to make sure data is saved in the database table
# # connection.commit()

In [ ]:
# # Retrieve all data from the table
# cursor.execute("SELECT * FROM articles_data")

# # Print each table row
# for row in cursor:
#     print(row)